Exercise: train and fine-tune a Decision Tree for the moons dataset.
---




In [37]:
from sklearn.datasets import make_moons
import numpy as np


In [38]:
X, y = make_moons(n_samples=10000, noise=0.4)

In [39]:
from sklearn.tree import DecisionTreeClassifier
desc_tree_clfr = DecisionTreeClassifier()

In [40]:
#Split Data 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)


In [41]:
from sklearn.model_selection import GridSearchCV
parameters = {
    "criterion" : ["gini", "entropy"],
    "max_depth" : [3,4,5,6,7,10,15,25,30,200], 
    "min_samples_split" : [50,100,250,500],
    "min_samples_leaf" : [1,10,150,250,500,1000], 
    "max_features" : [1,2],

}

gridsearch = GridSearchCV(desc_tree_clfr, parameters, cv=3)


In [42]:
gridsearch.fit(x_train, y_train)

GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=None,
                                              splitter='best'),
             iid='deprecated', n_jobs=None,
             param_grid={'criterion': ['gini', 'entropy'],
                   

In [43]:
gridsearch.best_params_

{'criterion': 'entropy',
 'max_depth': 10,
 'max_features': 1,
 'min_samples_leaf': 10,
 'min_samples_split': 100}

In [44]:
gridsearch.best_score_

0.8607493549427851

In [45]:
from sklearn.metrics import accuracy_score
y_pred = gridsearch.predict(x_test)
accuracy_score(y_test, y_pred)

0.866

Exercise: Grow a Forest.


a. Continuing the previous exercise, generate 1,000 subsets of the training set, each containing 100 instances selected randomly. Hint: you can use Scikit-Learn's ShuffleSplit class for this.


In [46]:
from sklearn.model_selection import ShuffleSplit
n_trees = 1000
subsets = ShuffleSplit(n_splits=1000, test_size=0.2)

In [47]:
minisets = []
for train_sub_idx, test_sub_idx in subsets.split(x_train): #returnn index for each instance, for X returns 1 index, but 2 values in X_train

  X_mini_train = x_train[train_sub_idx] #i.e. x_train[0] return 2 values (two features)
  y_mini_train = y_train[train_sub_idx]#i.e. y_train[0] return 1 value (label)
  minisets.append((X_mini_train, y_mini_train)) #list of datasets

In [48]:
from sklearn.base import clone  
forest = [clone(gridsearch.best_estimator_) for _ in range(n_trees)]

In [49]:
accuracy_scores = []
for tree, (mini_x_subset, mini_y_subset) in zip(forest, minisets):
  tree.fit(mini_x_subset,mini_y_subset)
  y_predictions=tree.predict(x_test)
  accuracy_scores.append(accuracy_score(y_test,y_predictions))



In [50]:
np.mean(accuracy_scores)

0.8580840000000001

In [51]:
y_predi = np.empty([n_trees, len(x_test)])
for idx, tree in enumerate(forest):
  y_predi[idx] = tree.predict(x_test)

In [52]:
from scipy.stats import mode
y_pred_majority_votes, n_votes = mode(y_predi, axis=0)

In [53]:
y_pred_majority_votes, n_votes

(array([[0., 0., 0., ..., 1., 0., 1.]]),
 array([[1000,  963,  997, ..., 1000, 1000, 1000]]))

In [54]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.866